# Quick Sample Flow - All Models Demo

This section demonstrates a quick sample run of all implemented model classes with minimal data and iterations to verify everything works correctly.

## Quick Setup - Load Minimal Data

In [1]:
import os
import sys
import numpy as np
from PIL import Image
from copy import deepcopy

# Add parent directory to path to import our models
sys.path.append(os.path.join(os.path.curdir, ".."))

# Dataset directories
DATASET_DIR = os.path.join(os.path.curdir, "..", ".cache", "processed_datasets")
CIFAR10_PATH = os.path.join(DATASET_DIR, "cifar10")
MNIST_PATH = os.path.join(DATASET_DIR, "mnist")

print("Quick sample flow setup completed!")
print(f"Dataset directory: {DATASET_DIR}")
print(f"CIFAR-10 path: {CIFAR10_PATH}")
print(f"MNIST path: {MNIST_PATH}")

Quick sample flow setup completed!
Dataset directory: .\..\.cache\processed_datasets
CIFAR-10 path: .\..\.cache\processed_datasets\cifar10
MNIST path: .\..\.cache\processed_datasets\mnist


In [2]:
# Load datasets (small sample for quick demo)
import warnings
warnings.filterwarnings('ignore')

from datasets import load_from_disk

try:
    # Load datasets from cache
    cifar10_dataset = load_from_disk(CIFAR10_PATH)
    mnist_dataset = load_from_disk(MNIST_PATH)
    
    # Take small samples for quick demo (100 samples each)
    SAMPLE_SIZE = 100
    
    # Sample from CIFAR-10
    cifar10_sample = cifar10_dataset['train'].shuffle(seed=42).select(range(SAMPLE_SIZE))
    cifar10_test_sample = cifar10_dataset['test'].shuffle(seed=42).select(range(50))
    
    # Sample from MNIST
    mnist_sample = mnist_dataset['train'].shuffle(seed=42).select(range(SAMPLE_SIZE))
    mnist_test_sample = mnist_dataset['test'].shuffle(seed=42).select(range(50))
    
    print("Sample datasets loaded successfully!")
    print(f"CIFAR-10 sample: {len(cifar10_sample)} train, {len(cifar10_test_sample)} test")
    print(f"MNIST sample: {len(mnist_sample)} train, {len(mnist_test_sample)} test")
    
except Exception as e:
    print(f"Error loading datasets: {e}")
    print("Please make sure datasets are downloaded and processed first.")

Sample datasets loaded successfully!
CIFAR-10 sample: 100 train, 50 test
MNIST sample: 100 train, 50 test


In [3]:
# Data extraction utility function
def extract_features_and_labels(dataset, flatten_images=True, max_samples=None):
    """Extract features (X) and labels (y) from a HuggingFace Dataset"""
    images = dataset['image']
    labels = dataset['label']
    
    # Limit samples if specified
    if max_samples is not None:
        images = images[:max_samples]
        labels = labels[:max_samples]
    
    # Convert images to numpy arrays
    X = []
    for img in images:
        if isinstance(img, Image.Image):
            img_array = np.array(img)
        else:
            img_array = img
        
        if flatten_images:
            img_array = img_array.flatten()
        
        X.append(img_array)
    
    X = np.array(X)
    y = np.array(labels)
    
    return X, y

# Extract sample data
X_cifar, y_cifar = extract_features_and_labels(cifar10_sample, flatten_images=True)
X_cifar_test, y_cifar_test = extract_features_and_labels(cifar10_test_sample, flatten_images=True)

X_mnist, y_mnist = extract_features_and_labels(mnist_sample, flatten_images=True)
X_mnist_test, y_mnist_test = extract_features_and_labels(mnist_test_sample, flatten_images=True)

print(f"CIFAR-10 sample shapes: X={X_cifar.shape}, y={y_cifar.shape}")
print(f"MNIST sample shapes: X={X_mnist.shape}, y={y_mnist.shape}")
print(f"Pixel value ranges - CIFAR-10: [{X_cifar.min()}, {X_cifar.max()}], MNIST: [{X_mnist.min()}, {X_mnist.max()}]")

CIFAR-10 sample shapes: X=(100, 1024), y=(100,)
MNIST sample shapes: X=(100, 1024), y=(100,)
Pixel value ranges - CIFAR-10: [0.0, 1.0], MNIST: [0.0, 1.0]


## Load All Model Classes

In [4]:
# Silence warnings to avoid printing full paths
import warnings
warnings.filterwarnings('ignore')

# Import all model classes
from models.decision_tree import DecisionTreeModel
from models.knn import KNNModel
from models.logistic_regression import LogisticRegressionModel
from models.mlp import MLPModel
from models.cnn import CNNModel

def create_sample_models():
    """Create instances of all model classes for testing"""
    models = {}
    
    # Decision Tree
    dt_model = DecisionTreeModel()
    dt_model.create_model()
    models["Decision Tree"] = dt_model
    
    # K-Nearest Neighbors
    knn_model = KNNModel()
    knn_model.create_model()
    models["K-Nearest Neighbors"] = knn_model
    
    # Logistic Regression (Logistic Regression for classification)
    lr_model = LogisticRegressionModel()
    lr_model.create_model()
    models["Logistic Regression"] = lr_model
    
    # Multi-Layer Perceptron
    mlp_model = MLPModel()
    mlp_model.create_model()
    models["Multi-Layer Perceptron"] = mlp_model
    
    # CNN (Note: May require special handling due to PyTorch)
    try:
        cnn_model = CNNModel()
        cnn_model.create_model()
        models["Convolutional Neural Network"] = cnn_model
    except Exception as e:
        print(f"[ERROR] CNN model creation failed: {e}")
        print("CNN will be skipped in quick demo")
    
    return models

# Create model instances
sample_models = create_sample_models()

print("Model classes loaded successfully!")
for name, model in sample_models.items():
    print(f"  {name}: {type(model).__name__}")
    
print(f"\nTotal models available: {len(sample_models)}")

Using Device: cpu
Is CUDA Available: False
CNN Model Architecture (input channels: 1):
CNNModel(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): AdaptiveAvgPool2d(output_size=1)
  )
  (model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Dropout(p=0.5, inplace=False)
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)
Model classes loaded successfully!
  Decision Tree: DecisionTreeModel
  K-Nearest N

## Quick Hyperparameter Testing

In [5]:
import itertools
import random

def get_class_names_from_dataset(dataset_name):
    """Extract class names from HuggingFace dataset features"""
    try:
        if dataset_name == "MNIST":
            # Try to get from MNIST dataset features
            if hasattr(mnist_dataset['train'].features['label'], 'names') and mnist_dataset['train'].features['label'].names:
                return mnist_dataset['train'].features['label'].names
            else:
                # MNIST typically uses digits 0-9
                return [str(i) for i in range(10)]
        elif dataset_name == "CIFAR-10":
            # Try to get from CIFAR-10 dataset features
            if hasattr(cifar10_dataset['train'].features['label'], 'names') and cifar10_dataset['train'].features['label'].names:
                return cifar10_dataset['train'].features['label'].names
            else:
                # CIFAR-10 fallback class names
                return ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
        else:
            # For other datasets, create generic class names based on unique labels
            return [f'Class {i}' for i in range(10)]  # Default to 10 classes
    except Exception as e:
        print(f"Warning: Could not extract class names for {dataset_name}: {e}")
        # Fallback to generic class names
        if dataset_name == "MNIST":
            return [str(i) for i in range(10)]
        elif dataset_name == "CIFAR-10":
            return ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
        else:
            return [f'Class {i}' for i in range(10)]

def quick_hyperparameter_test(models_dict, X_train, y_train, X_test, y_test, dataset_name="Dataset"):
    """Perform a quick hyperparameter test with limited iterations"""
    print(f"Starting quick hyperparameter test on {dataset_name}")
    print("="*60)
    
    # Extract class names from HuggingFace dataset - ALWAYS get class names for CNN compatibility
    class_names = get_class_names_from_dataset(dataset_name)
    print(f"Extracted class names from {dataset_name}: {class_names}")
    
    results = {}
    
    for model_name, model in models_dict.items():
        print(f"\nTesting {model_name}...")
        
        # Check if model supports hyperparameter tuning
        if not hasattr(model, 'get_param_space'):
            print(f"[WARNING] {model_name} does not support hyperparameter tuning. Using default params.")
            try:
                model_copy = deepcopy(model)
                # CNN models ALWAYS need class_names (required positional argument)
                if 'CNN' in model_name:
                    model_copy.train(X_train, y_train, class_names)
                else:
                    model_copy.train(X_train, y_train)
                metrics = model_copy.evaluate(X_test, y_test)
                results[model_name] = {
                    'best_params': 'default',
                    'best_score': metrics.get('accuracy', 0.0),
                    'metrics': metrics
                }
                print(f"Default accuracy: {metrics.get('accuracy', 0.0):.4f}")
            except Exception as e:
                print(f"[ERROR] {e}")
                results[model_name] = {'error': str(e)}
            continue
        
        # Get parameter space and sample a few combinations
        try:
            param_space = model.get_param_space()
            param_names = list(param_space.keys())
            
            # Extract actual values from ParamSpace objects - helper function defined in comprehensive section
            def extract_param_values_for_quick_test(param_space):
                """Quick version of param extraction with fewer samples"""
                param_values = []
                for param_name, param_def in param_space.items():
                    if hasattr(param_def, 'param_type'):
                        # This is a ParamSpace object
                        if param_def.param_type.value == 'categorical':
                            param_values.append(param_def.choices)
                        elif param_def.param_type.value == 'boolean':
                            param_values.append([True, False])
                        elif param_def.param_type.value == 'integer':
                            # Sample a few values from the range for quick testing
                            values = []
                            if param_def.min_value is not None and param_def.max_value is not None:
                                step = max(1, (param_def.max_value - param_def.min_value) // 3)
                                values = list(range(param_def.min_value, param_def.max_value + 1, step))
                                if len(values) > 5:  # Limit to 5 values for quick testing
                                    values = values[:5]
                            if param_def.default is not None and param_def.default not in values:
                                values.append(param_def.default)
                            param_values.append(values if values else [param_def.default])
                        elif param_def.param_type.value == 'float':
                            # Sample a few values from the range for quick testing
                            values = []
                            if param_def.min_value is not None and param_def.max_value is not None:
                                import numpy as np
                                values = list(np.linspace(param_def.min_value, param_def.max_value, 3))
                            if param_def.default is not None and param_def.default not in values:
                                values.append(param_def.default)
                            param_values.append(values if values else [param_def.default])
                    else:
                        # This is already a list of values
                        param_values.append(param_def)
                
                return param_values
            
            param_values = extract_param_values_for_quick_test(param_space)
            
            # Generate all combinations and sample a few
            all_combinations = list(itertools.product(*param_values))
            max_test = min(3, len(all_combinations))  # Test max 3 combinations for speed
            test_combinations = random.sample(all_combinations, max_test) if len(all_combinations) > max_test else all_combinations
            
            print(f"Testing {len(test_combinations)}/{len(all_combinations)} parameter combinations...")
            
            best_score = -1
            best_params = None
            best_metrics = None
            
            for i, param_combo in enumerate(test_combinations):
                current_params = dict(zip(param_names, param_combo))
                
                try:
                    # Create fresh model copy
                    model_copy = deepcopy(model)
                    
                    # Set parameters - handle both sklearn and PyTorch models
                    if hasattr(model_copy.model, 'set_params'):
                        # sklearn models
                        model_copy.model.set_params(**current_params)
                    elif hasattr(model_copy, 'set_params'):
                        # Custom Models like PyTorch
                        model_copy.set_params(**current_params)
                    else:
                        print(f"Combo {i+1}: ERROR - Model does not support parameter setting")
                        continue
                    
                    # Train and evaluate - CNN models ALWAYS need class_names
                    if 'CNN' in model_name:
                        model_copy.train(X_train, y_train, class_names)
                    else:
                        model_copy.train(X_train, y_train)
                    metrics = model_copy.evaluate(X_test, y_test)
                    accuracy = metrics.get('accuracy', 0.0)
                    
                    if accuracy > best_score:
                        best_score = accuracy
                        best_params = current_params.copy()
                        best_metrics = metrics.copy()
                    
                    print(f"Combo {i+1}: accuracy={accuracy:.4f}")
                    
                except Exception as e:
                    print(f"Combo {i+1}: ERROR - {e}")
            
            results[model_name] = {
                'best_params': best_params,
                'best_score': best_score,
                'metrics': best_metrics
            }
            
            print(f"Best accuracy: {best_score:.4f}")
            
        except Exception as e:
            print(f"[ERROR] Hyperparameter Tuning failed: {e}")
            results[model_name] = {'error': str(e)}
    
    return results

# Test on MNIST (smaller images, easier for quick demo)
print("Testing all models on MNIST sample...")
mnist_results = quick_hyperparameter_test(sample_models, X_mnist, y_mnist, X_mnist_test, y_mnist_test, "MNIST")

print(f"\n" + "="*80)
print("QUICK TEST RESULTS SUMMARY - MNIST")
print("="*80)
for model_name, result in mnist_results.items():
    if 'error' in result:
        print(f"[ERROR] {model_name} failed - {result['error']}")
    else:
        print(f"[RESULT] {model_name}: Accuracy = {result['best_score']:.4f}")

Testing all models on MNIST sample...
Starting quick hyperparameter test on MNIST
Extracted class names from MNIST: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

Testing Decision Tree...
Testing 3/500 parameter combinations...
Combo 1: accuracy=0.3400
Combo 2: accuracy=0.4200
Combo 3: accuracy=0.3400
Best accuracy: 0.4200

Testing K-Nearest Neighbors...
Testing 3/240 parameter combinations...
Combo 1: accuracy=0.6400
Combo 2: accuracy=0.6400
Combo 3: accuracy=0.6400
Best accuracy: 0.6400

Testing Logistic Regression...
Testing 3/100 parameter combinations...
Combo 1: accuracy=0.7400
Combo 2: accuracy=0.7200
Combo 3: accuracy=0.6000
Best accuracy: 0.7400

Testing Multi-Layer Perceptron...
Testing 3/540 parameter combinations...
Combo 1: accuracy=0.5600
Combo 2: accuracy=0.7000
Combo 3: accuracy=0.7400
Best accuracy: 0.7400

Testing Convolutional Neural Network...
Testing 3/54000 parameter combinations...
CNN Model Architecture (input channels: 1):
CNNModel(
  (features): Sequentia

Train Loss: 2.3228, Train Acc: 0.0625 (6.25%)
Val Loss: 2.3211, Val Acc: 0.1000 (10.00%)
Saved best model (val_acc=0.1000) to .cache/models/cnn_cifar.pth
Epoch [1/10], Train Loss: 2.3228, Val Loss: 2.3211

Epoch 2/10


Train Loss: 2.1861, Train Acc: 0.1500 (15.00%)
Val Loss: 2.3169, Val Acc: 0.1000 (10.00%)
Epoch [2/10], Train Loss: 2.1861, Val Loss: 2.3169

Epoch 3/10


Train Loss: 1.7558, Train Acc: 0.6375 (63.75%)
Val Loss: 2.3019, Val Acc: 0.1000 (10.00%)
Epoch [3/10], Train Loss: 1.7558, Val Loss: 2.3019

Epoch 4/10


Train Loss: 1.2977, Train Acc: 0.8500 (85.00%)
Val Loss: 2.2680, Val Acc: 0.1000 (10.00%)
Epoch [4/10], Train Loss: 1.2977, Val Loss: 2.2680

Epoch 5/10


...

Train Loss: 1.1432, Train Acc: 0.9500 (95.00%)
Val Loss: 2.1577, Val Acc: 0.4500 (45.00%)
Saved best model (val_acc=0.4500) to .cache/models/cnn_cifar.pth
Epoch [6/10], Train Loss: 1.1432, Val Loss: 2.1577

Epoch 7/10


Train Loss: 1.0973, Train Acc: 0.9375 (93.75%)
Val Loss: 2.1012, Val Acc: 0.5500 (55.00%)
Saved best model (val_acc=0.5500) to .cache/models/cnn_cifar.pth
Epoch [7/10], Train Loss: 1.0973, Val Loss: 2.1012

Epoch 8/10


Train Loss: 0.9620, Train Acc: 0.9750 (97.50%)
Val Loss: 2.0395, Val Acc: 0.6500 (65.00%)
Saved best model (val_acc=0.6500) to .cache/models/cnn_cifar.pth
Epoch [8/10], Train Loss: 0.9620, Val Loss: 2.0395

Epoch 9/10


Train Loss: 0.9829, Train Acc: 0.9750 (97.50%)
Val Loss: 1.9720, Val Acc: 0.6500 (65.00%)
Epoch [9/10], Train Loss: 0.9829, Val Loss: 1.9720

Epoch 10/10


Train Loss: 0.9458, Train Acc: 1.0000 (100.00%)
Val Loss: 1.9026, Val Acc: 0.7000 (70.00%)
Saved best model (val_acc=0.7000) to .cache/models/cnn_cifar.pth
Epoch [10/10], Train Loss: 0.9458, Val Loss: 1.9026

Training complete!
Best val acc: 0.7000 (70.00%)
Training history plot saved to: .cache\results\cnn_training_history_20251111_181100.png
Training history data saved to: .cache\results\cnn_training_history_20251111_181100_data.txt
Confusion matrices saved to: .cache\results\cnn_confusion_matrices_20251111_181100.png
Confusion matrix data saved to: .cache\results\cnn_confusion_matrices_20251111_181100_data.txt
Preprocessing input data for test data...
Original shape: (50, 1024)
Detected 32x32 grayscale format: reshaped to (50, 32, 32, 1)
Final tensor shape: torch.Size([50, 1, 32, 32]) (N, C, H, W)
Preprocessing input data for test data...
Original shape: (50, 1024)
Detected 32x32 grayscale format: reshaped to (50, 32, 32, 1)
Final tensor shape: torch.Size([50, 1, 32, 32]) (N, C, H, 

In [6]:
# Test on CIFAR-10 as well
print("\n" + "="*60)
print("Testing all models on CIFAR-10 sample...")
cifar_results = quick_hyperparameter_test(sample_models, X_cifar, y_cifar, X_cifar_test, y_cifar_test, "CIFAR-10")

print(f"\n" + "="*80)
print("QUICK TEST RESULTS SUMMARY - CIFAR-10")  
print("="*80)
for model_name, result in cifar_results.items():
    if 'error' in result:
        print(f"[ERROR] {model_name} failed - {result['error']}")
    else:
        print(f"[RESULT] {model_name}: Accuracy = {result['best_score']:.4f}")


Testing all models on CIFAR-10 sample...
Starting quick hyperparameter test on CIFAR-10
Extracted class names from CIFAR-10: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

Testing Decision Tree...
Testing 3/500 parameter combinations...
Combo 1: accuracy=0.1600
Combo 2: accuracy=0.1000
Combo 3: accuracy=0.1400
Best accuracy: 0.1600

Testing K-Nearest Neighbors...
Testing 3/240 parameter combinations...
Combo 1: accuracy=0.1000
Combo 2: accuracy=0.0800
Combo 3: accuracy=0.1200
Best accuracy: 0.1200

Testing Logistic Regression...
Testing 3/100 parameter combinations...
Combo 1: accuracy=0.1000
Combo 2: accuracy=0.0800
Combo 3: accuracy=0.0800
Best accuracy: 0.1000

Testing Multi-Layer Perceptron...
Testing 3/540 parameter combinations...
Combo 1: accuracy=0.1400
Combo 2: accuracy=0.1000
Combo 3: accuracy=0.1200
Best accuracy: 0.1400

Testing Convolutional Neural Network...
Testing 3/54000 parameter combinations...
CNN Model Architecture (inp

Train Loss: 2.3503, Train Acc: 0.1000 (10.00%)
Val Loss: 2.3149, Val Acc: 0.0000 (0.00%)
Epoch [1/10], Train Loss: 2.3503, Val Loss: 2.3149

Epoch 2/10


Train Loss: 2.3034, Train Acc: 0.1000 (10.00%)
Val Loss: 2.3253, Val Acc: 0.0000 (0.00%)
Epoch [2/10], Train Loss: 2.3034, Val Loss: 2.3253

Epoch 3/10


Train Loss: 2.2346, Train Acc: 0.1875 (18.75%)
Val Loss: 2.3314, Val Acc: 0.0000 (0.00%)
Epoch [3/10], Train Loss: 2.2346, Val Loss: 2.3314

Epoch 4/10


Train Loss: 2.1996, Train Acc: 0.1625 (16.25%)
Val Loss: 2.3125, Val Acc: 0.0500 (5.00%)
Saved best model (val_acc=0.0500) to .cache/models/cnn_cifar.pth
Epoch [4/10], Train Loss: 2.1996, Val Loss: 2.3125

Epoch 5/10


...

Train Loss: 2.0153, Train Acc: 0.2500 (25.00%)
Val Loss: 2.2643, Val Acc: 0.2500 (25.00%)
Epoch [6/10], Train Loss: 2.0153, Val Loss: 2.2643

Epoch 7/10


Train Loss: 1.9595, Train Acc: 0.2875 (28.75%)
Val Loss: 2.2672, Val Acc: 0.2500 (25.00%)
Epoch [7/10], Train Loss: 1.9595, Val Loss: 2.2672

Epoch 8/10


Train Loss: 1.9803, Train Acc: 0.3375 (33.75%)
Val Loss: 2.2759, Val Acc: 0.2500 (25.00%)
Epoch [8/10], Train Loss: 1.9803, Val Loss: 2.2759

Epoch 9/10


Train Loss: 1.8654, Train Acc: 0.3875 (38.75%)
Val Loss: 2.2838, Val Acc: 0.2500 (25.00%)
Epoch [9/10], Train Loss: 1.8654, Val Loss: 2.2838

Epoch 10/10


Train Loss: 1.9232, Train Acc: 0.3375 (33.75%)
Val Loss: 2.2841, Val Acc: 0.2500 (25.00%)
Epoch [10/10], Train Loss: 1.9232, Val Loss: 2.2841

Training complete!
Best val acc: 0.2500 (25.00%)
Training history plot saved to: .cache\results\cnn_training_history_20251111_181120.png
Training history data saved to: .cache\results\cnn_training_history_20251111_181120_data.txt
Confusion matrices saved to: .cache\results\cnn_confusion_matrices_20251111_181121.png
Confusion matrix data saved to: .cache\results\cnn_confusion_matrices_20251111_181121_data.txt
Preprocessing input data for test data...
Original shape: (50, 1024)
Detected 32x32 grayscale format: reshaped to (50, 32, 32, 1)
Final tensor shape: torch.Size([50, 1, 32, 32]) (N, C, H, W)
Preprocessing input data for test data...
Original shape: (50, 1024)
Detected 32x32 grayscale format: reshaped to (50, 32, 32, 1)
Final tensor shape: torch.Size([50, 1, 32, 32]) (N, C, H, W)
Combo 3: accuracy=0.1200
Best accuracy: 0.1200

QUICK TEST RESU

Noticing the exceptionally low scores, it could be the reason of the **quick** test. It only uses 100 training samples to quickly verify whether the pipeline works normally. So now, we go ahead with a more detailed flow.

## Model Interface Verification

In [7]:
# Verify that all models implement the required interface correctly
def verify_model_interface(model_dict):
    """Verify that all models implement the BaseModel interface correctly"""
    print("Verifying model interfaces...")
    print("="*50)
    
    for name, model in model_dict.items():
        print(f"\n{name}:")
        
        # Check required methods
        required_methods = ['create_model', 'get_param_space', 'train', 'predict', 'evaluate']
        missing_methods = []
        
        for method in required_methods:
            if hasattr(model, method):
                print(f"{method}()")
            else:
                print(f"[WARNING] {method}() - MISSING")
                missing_methods.append(method)
        
        # Check parameter space
        try:
            if hasattr(model, 'get_param_space'):
                param_space = model.get_param_space()
                print(f"Parameter space: {len(param_space)} parameters")
                for param_name, param_def in param_space.items():
                    print(f"- {param_name}: {param_def.param_type.value}")
            else:
                print("[WARNING] No parameter space available")
        except Exception as e:
            print(f"[WARNING] Parameter space error: {e}")

        # Check if model is created
        if hasattr(model, 'model') and model.model is not None:
            print(f"Model instance: {type(model.model).__name__}")
        else:
            print("No model instance found")

        if missing_methods:
            print(f"[WARNING] INTERFACE INCOMPLETE: Missing {missing_methods}")
        else:
            print(f"[RESULT] INTERFACE COMPLETE")

# Run interface verification
verify_model_interface(sample_models)

Verifying model interfaces...

Decision Tree:
create_model()
get_param_space()
train()
predict()
evaluate()
Parameter space: 5 parameters
- max_depth: integer
- min_samples_split: integer
- min_samples_leaf: integer
- criterion: categorical
- splitter: categorical
Model instance: DecisionTreeClassifier
[RESULT] INTERFACE COMPLETE

K-Nearest Neighbors:
create_model()
get_param_space()
train()
predict()
evaluate()
Parameter space: 5 parameters
- n_neighbors: integer
- weights: categorical
- algorithm: categorical
- p: categorical
- metric: categorical
Model instance: KNeighborsClassifier
[RESULT] INTERFACE COMPLETE

Logistic Regression:
create_model()
get_param_space()
train()
predict()
evaluate()
Parameter space: 4 parameters
- C: categorical
- penalty: categorical
- solver: categorical
- l1_ratio: categorical
Model instance: LogisticRegression
[RESULT] INTERFACE COMPLETE

Multi-Layer Perceptron:
create_model()
get_param_space()
train()
predict()
evaluate()
Parameter space: 5 parameters

## Quick Demo Summary

This quick sample flow demonstrates:

1. **All Model Classes Loaded**: Successfully imported and instantiated all 5 model classes from the `models/` directory
2. **Interface Compliance**: Verified that all models implement the required `BaseModel` interface
3. **Hyperparameter Testing**: Tested hyperparameter tuning functionality with small sample data
4. **Training & Evaluation**: Confirmed that all models can train and evaluate on both MNIST and CIFAR-10 data

### Models Tested:
- ✅ **Decision Tree Model** (`DecisionTreeModel`)
- ✅ **K-Nearest Neighbors Model** (`KNNModel`) 
- ✅ **Logistic Regression Model** (`LogisticRegressionModel`)
- ✅ **Multi-Layer Perceptron Model** (`MLPModel`)
- ✅ **Convolutional Neural Network Model** (`CNNModel`)

This quick flow uses small sample sizes (100 training, 50 test samples) and limited hyperparameter combinations (max 3 per model) to ensure fast execution while still validating that the complete machine learning pipeline works correctly for all implemented model classes.

---

**Note**: For full experiments, use the comprehensive workflow sections below with complete datasets and extensive hyperparameter search.

## Model Interface Verification

# Establishing an Ordinary Model Training Workflow

## Load the Data

In [8]:
import os
DATASET_DIR = os.path.join(
    os.path.curdir,
    "..",
    ".cache",
    "processed_datasets"
)
CIFAR10_PATH = os.path.join(DATASET_DIR, "cifar10")
MNIST_PATH = os.path.join(DATASET_DIR, "mnist")

In [9]:
# Load from HuggingFace datasets
from datasets import load_from_disk

# Load all datasets from cache
cifar10_dataset = load_from_disk(CIFAR10_PATH)
mnist_dataset = load_from_disk(MNIST_PATH)

# Access train and test splits for all datasets
CIFAR10_TRAIN = cifar10_dataset['train']
CIFAR10_TEST = cifar10_dataset['test']
MNIST_TRAIN = mnist_dataset['train']
MNIST_TEST = mnist_dataset['test']

# Display dataset information
print("Loaded datasets from cache:")
print(f"CIFAR-10 Train: {len(CIFAR10_TRAIN):,} examples")
print(f"CIFAR-10 Test: {len(CIFAR10_TEST):,} examples")
print(f"MNIST Train: {len(MNIST_TRAIN):,} examples")
print(f"MNIST Test: {len(MNIST_TEST):,} examples")
print(f"\nCIFAR-10 classes: {CIFAR10_TRAIN.features['label'].names}")
print(f"MNIST classes: {list(range(10))}")  # MNIST has digits 0-9

Loaded datasets from cache:
CIFAR-10 Train: 50,000 examples
CIFAR-10 Test: 10,000 examples
MNIST Train: 60,000 examples
MNIST Test: 10,000 examples

CIFAR-10 classes: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
MNIST classes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


## Prepare a Validation Set

In [10]:
# Split using HuggingFace datasets train_test_split method
cifar10_split = CIFAR10_TRAIN.train_test_split(test_size=500, seed=42)
CIFAR10_TRAIN = cifar10_split['train']
CIFAR10_VAL = cifar10_split['test']

mnist_split = MNIST_TRAIN.train_test_split(test_size=500, seed=42)
MNIST_TRAIN = mnist_split['train']
MNIST_VAL = mnist_split['test']

# Inspect the sizes
print("After splitting into Train and Validation sets:")
print(f"CIFAR-10 Train: {len(CIFAR10_TRAIN):,} examples")
print(f"CIFAR-10 Validation: {len(CIFAR10_VAL):,} examples")
print(f"MNIST Train: {len(MNIST_TRAIN):,} examples")
print(f"MNIST Validation: {len(MNIST_VAL):,} examples")

After splitting into Train and Validation sets:
CIFAR-10 Train: 49,500 examples
CIFAR-10 Validation: 500 examples
MNIST Train: 59,500 examples
MNIST Validation: 500 examples


## Get the Machine Learning Models

In [11]:
import sys
sys.path.append(os.path.join(os.path.curdir, ".."))

from models.decision_tree import DecisionTreeModel
from models.knn import KNNModel
from models.logistic_regression import LogisticRegressionModel
from models.mlp import MLPModel
from models.cnn import CNNModel

# Instantiate model classes (not the sklearn models directly)
dt_model = DecisionTreeModel()
dt_model.create_model()
knn_model = KNNModel()
knn_model.create_model()
lr_model = LogisticRegressionModel()
lr_model.create_model()
mlp_model = MLPModel()
mlp_model.create_model()
cnn_model = CNNModel()
cnn_model.create_model()

# Display those model instances
models = {
    "Decision Tree Model": dt_model,
    "K-Nearest Neighbors Model": knn_model,
    "Logistic Regression Model": lr_model,
    "Multi-Layer Perceptron Model": mlp_model,
    "Convolutional Neural Network Model": cnn_model
}

# Show the model classes
for name, model in models.items():
    print(f"{name}: {type(model).__name__}")
    
models

CNN Model Architecture (input channels: 1):
CNNModel(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): AdaptiveAvgPool2d(output_size=1)
  )
  (model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Dropout(p=0.5, inplace=False)
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)
Decision Tree Model: DecisionTreeModel
K-Nearest Neighbors Model: KNNModel
Logistic Regression Model: LogisticRegressionModel


{'Decision Tree Model': <models.decision_tree.DecisionTreeModel at 0x1ff5ceb7da0>,
 'K-Nearest Neighbors Model': <models.knn.KNNModel at 0x1ff5ceb6bd0>,
 'Logistic Regression Model': <models.logistic_regression.LogisticRegressionModel at 0x1ff5dfdbf80>,
 'Multi-Layer Perceptron Model': <models.mlp.MLPModel at 0x1ff57a0c530>,
 'Convolutional Neural Network Model': CNNModel(
   (features): Sequential(
     (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
     (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): ReLU(inplace=True)
     (3): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
     (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (5): ReLU(inplace=True)
     (6): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
     (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (8): ReLU(inplace=True)
     (9): A

## Tune the Hyperparameters

We tune it with our own multi-objective fitness functions across different metaheuristics. For simlicity, let's just demonstrate how we weigh the importances of every metric.

In [12]:
import numpy as np

# Note: extract_features_and_labels is defined in the quick demo section above with max_samples parameter. We are reusing it here.

# Extract validation features and labels for CIFAR-10
X_CIFAR10_VAL, y_CIFAR10_VAL = extract_features_and_labels(CIFAR10_VAL, flatten_images=True)
print(f"CIFAR-10 Validation:")
print(f"X_VAL shape: {X_CIFAR10_VAL.shape}")
print(f"y_VAL shape: {y_CIFAR10_VAL.shape}")
print(f"Data type: X={X_CIFAR10_VAL.dtype}, y={y_CIFAR10_VAL.dtype}")

# Extract validation features and labels for MNIST
X_MNIST_VAL, y_MNIST_VAL = extract_features_and_labels(MNIST_VAL, flatten_images=True)
print(f"\nMNIST Validation:")
print(f"X_VAL shape: {X_MNIST_VAL.shape}")
print(f"y_VAL shape: {y_MNIST_VAL.shape}")
print(f"Data type: X={X_MNIST_VAL.dtype}, y={y_MNIST_VAL.dtype}")

# Show label examples
print(f"\nCIFAR-10 label examples: {y_CIFAR10_VAL[:5]}")
print(f"MNIST label examples: {y_MNIST_VAL[:5]}")
print(f"Pixel value range CIFAR-10: [{X_CIFAR10_VAL.min()}, {X_CIFAR10_VAL.max()}]")
print(f"Pixel value range MNIST: [{X_MNIST_VAL.min()}, {X_MNIST_VAL.max()}]")

CIFAR-10 Validation:
X_VAL shape: (500, 1024)
y_VAL shape: (500,)
Data type: X=float32, y=int64

MNIST Validation:
X_VAL shape: (500, 1024)
y_VAL shape: (500,)
Data type: X=float32, y=int64

CIFAR-10 label examples: [1 2 6 7 9]
MNIST label examples: [8 7 1 6 0]
Pixel value range CIFAR-10: [0.0, 1.0]
Pixel value range MNIST: [0.0, 1.0]


The cell below shows 3 alternative methods of extracting the images array and the labels array, just as a reference to check whether the above extraction behaves correctly. 

In [13]:
# Alternative methods for extracting X and y

# Method 1: Direct column access (simpler but less flexible)
print("=== Alternative Method 1: Direct Column Access ===")
y_cifar_simple = np.array(CIFAR10_VAL['label'])
print(f"y_CIFAR10_VAL shape: {y_cifar_simple.shape}")

# Method 2: Using dataset.to_pandas()
print("\n=== Alternative Method 2: Using to_pandas() ===")
cifar_df = CIFAR10_VAL.to_pandas()
print(f"DataFrame shape: {cifar_df.shape}")
print(f"DataFrame columns: {list(cifar_df.columns)}")

# Method 3: Batch processing for large datasets (memory efficient)
print("\n=== Alternative Method 3: Batch Processing ===")
def extract_in_batches(dataset, batch_size=1000, flatten_images=True):
    """Extract features and labels in batches to save memory"""
    total_samples = len(dataset)
    X_batches = []
    y_batches = []
    
    for i in range(0, total_samples, batch_size):
        batch = dataset[i:i+batch_size]
        
        # Process batch
        batch_images = []
        for img in batch['image']:
            img_array = np.array(img)
            if flatten_images:
                img_array = img_array.flatten()
            batch_images.append(img_array)
        
        X_batches.append(np.array(batch_images))
        y_batches.append(np.array(batch['label']))
    
    # Concatenate all batches
    X = np.vstack(X_batches)
    y = np.concatenate(y_batches)
    
    return X, y

# Example with small batch for demonstration
X_batch, y_batch = extract_in_batches(CIFAR10_VAL.select(range(100)), batch_size=50)
print(f"Batch extraction example - X shape: {X_batch.shape}, y shape: {y_batch.shape}")

=== Alternative Method 1: Direct Column Access ===
y_CIFAR10_VAL shape: (500,)

=== Alternative Method 2: Using to_pandas() ===
DataFrame shape: (500, 2)
DataFrame columns: ['image', 'label']

=== Alternative Method 3: Batch Processing ===
Batch extraction example - X shape: (100, 1024), y shape: (100,)


### Tuning the Hyperparameters by training on only **validation set** iteratively on each dataset and each model.

In [14]:
import itertools
from copy import deepcopy
import os
import time
import random

def calculate_weighted_score(metrics_dict):
    """Calculate weighted sum of evaluation metrics"""
    weighted_sum = 0.0
    for metric_name, score in metrics_dict.items():
        if metric_name in metrics_dict:
            weighted_sum += score * 1/len(metrics_dict) # Equal weights for simplicity
    weighted_sum *= 0.8 # avoid going to 1
    return weighted_sum

def get_class_names_from_dataset(dataset_name):
    """Extract class names from HuggingFace dataset features with reliable fallbacks"""
    try:
        if dataset_name == "MNIST":
            # Try to get from MNIST dataset features
            if hasattr(MNIST_TRAIN.features['label'], 'names') and MNIST_TRAIN.features['label'].names:
                return MNIST_TRAIN.features['label'].names
            else:
                # MNIST typically uses digits 0-9
                return [str(i) for i in range(10)]
        elif dataset_name == "CIFAR-10":
            # Try to get from CIFAR-10 dataset features
            if hasattr(CIFAR10_TRAIN.features['label'], 'names') and CIFAR10_TRAIN.features['label'].names:
                return CIFAR10_TRAIN.features['label'].names
            else:
                # CIFAR-10 fallback class names
                return ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
        else:
            # For other datasets, create generic class names
            return [f'Class {i}' for i in range(10)]  # Default to 10 classes
    except Exception as e:
        print(f"Warning: Could not extract class names for {dataset_name}: {e}")
        # Reliable fallbacks
        if dataset_name == "MNIST":
            return [str(i) for i in range(10)]
        elif dataset_name == "CIFAR-10":
            return ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
        else:
            return [f'Class {i}' for i in range(10)]

# Since ParamSpace is a custom class, we need to extract actual values with a helper function
def extract_param_values_from_param_space(param_space):
    """Extract actual values from ParamSpace objects for hyperparameter tuning"""
    param_values = []
    for param_name, param_def in param_space.items():
        if hasattr(param_def, 'param_type'):
            # This is a ParamSpace object
            if param_def.param_type.value == 'categorical':
                param_values.append(param_def.choices)
            elif param_def.param_type.value == 'boolean':
                param_values.append([True, False])
            elif param_def.param_type.value == 'integer':
                # Sample a few values from the range for comprehensive testing
                values = []
                if param_def.min_value is not None and param_def.max_value is not None:
                    step = max(1, (param_def.max_value - param_def.min_value) // 5)
                    values = list(range(param_def.min_value, param_def.max_value + 1, step))
                    if len(values) > 8:  # Limit to 8 values for comprehensive testing
                        values = values[:8]
                if param_def.default is not None and param_def.default not in values:
                    values.append(param_def.default)
                param_values.append(values if values else [param_def.default])
            elif param_def.param_type.value == 'float':
                # Sample a few values from the range for comprehensive testing
                values = []
                if param_def.min_value is not None and param_def.max_value is not None:
                    import numpy as np
                    values = list(np.linspace(param_def.min_value, param_def.max_value, 5))
                if param_def.default is not None and param_def.default not in values:
                    values.append(param_def.default)
                param_values.append(values if values else [param_def.default])
        else:
            # This is already a list of values
            param_values.append(param_def)
    
    return param_values

def _sanitize_paths_in_obj(obj, start=None):
    """Recursively convert absolute filesystem paths in strings to relative paths for safe printing."""
    if start is None:
        try:
            start = os.getcwd()
        except Exception:
            start = '.'

    # Strings: convert absolute paths to relative where possible
    if isinstance(obj, str):
        try:
            if os.path.isabs(obj):
                return os.path.relpath(obj, start)
        except Exception:
            return obj
        return obj

    # Dicts: recurse
    if isinstance(obj, dict):
        return {k: _sanitize_paths_in_obj(v, start) for k, v in obj.items()}

    # Lists / tuples: recurse and preserve type
    if isinstance(obj, (list, tuple)):
        converted = [_sanitize_paths_in_obj(v, start) for v in obj]
        return type(obj)(converted)

    # Other types: leave as-is
    return obj

# Storage for hyperparameter tuning results
tuning_results = {}
model_dataset_param = {}  # To Store "Best" Models

for dataset in ['CIFAR-10', 'MNIST']:
    for modelName, model in models.items():
        print(f"\n{'='*60}")
        print(f"Tuning hyperparameters for {modelName} on {dataset}")
        print(f"{'='*60}")
        
        # Get class names for this dataset - ALWAYS get them for CNN compatibility
        class_names = get_class_names_from_dataset(dataset)
        print(f"Using class names: {class_names}")
        
        # Get the appropriate validation data
        match dataset:
            case 'CIFAR-10':
                X_val, y_val = X_CIFAR10_VAL, y_CIFAR10_VAL
            case 'MNIST':
                X_val, y_val = X_MNIST_VAL, y_MNIST_VAL
        
        # Get parameter space
        if not hasattr(model, 'get_param_space'):
            print(f"[WARNING] Model {modelName} does not support hyperparameter tuning. Using default params.")
            try:
                model_copy = deepcopy(model)
                # CNN models ALWAYS need class_names (required positional argument)
                if 'CNN' in modelName:
                    model_copy.train(X_val, y_val, class_names)
                else:
                    model_copy.train(X_val, y_val)
                eval_metrics = model_copy.evaluate(X_val, y_val)
                weighted_score = calculate_weighted_score(eval_metrics)
                
                # Store results for models that don't support hyperparameter tuning
                tuning_results[(modelName, dataset)] = {
                    'best_params': 'default',
                    'best_score': weighted_score,
                    'best_metrics': eval_metrics,
                    'all_results': [{'params': 'default', 'metrics': eval_metrics, 'weighted_score': weighted_score}]
                }
                model_dataset_param[(modelName, dataset)] = model_copy
                print(f"Default weighted score: {weighted_score:.4f}")
                continue
            except Exception as e:
                print(f"[ERROR] Error with default params: {e}")
                tuning_results[(modelName, dataset)] = {'error': str(e)}
                model_dataset_param[(modelName, dataset)] = None
                continue
        
        param_space = model.get_param_space()
        param_names = list(param_space.keys())
        
        # Extract actual values from ParamSpace objects
        param_values = extract_param_values_from_param_space(param_space)
        
        # Generate all combinations of parameters
        param_combinations = list(itertools.product(*param_values))
        # Shuffle to randomize order because of iteration limit
        # Make sure every combination have an equal chance to be explored
        random.shuffle(param_combinations) 
        total_combinations = len(param_combinations)
        print(f"Testing {total_combinations} parameter combinations...")
        
        best_score = -1
        best_params = None
        best_model = None
        current_results = []
        
        # Iterate through all parameter combinations
        time_start = time.perf_counter()
        time_limit = 60 * 10 # 10 minutes
        iteration_count = 0
        iteration_limit = total_combinations * 0.6
        for i, param_combo in enumerate(param_combinations):
            # Check the Limits
            if time.perf_counter() - time_start > time_limit:
                print("Time limit reached, stopping further evaluations.")
                break
            if iteration_count >= iteration_limit:
                print("Iteration limit reached, stopping further evaluations.")
                break
            # Create parameter dictionary
            current_params = dict(zip(param_names, param_combo))
            
            # Create a fresh copy of the model for this configuration
            model_copy = deepcopy(model)
            
            # Set parameters - handle both sklearn and PyTorch models
            try:
                if hasattr(model_copy.model, 'set_params'):
                    # sklearn models
                    model_copy.model.set_params(**current_params)
                elif hasattr(model_copy, 'set_params'):
                    # Custom Models like PyTorch
                    model_copy.set_params(**current_params)
                else:
                    safe_params = _sanitize_paths_in_obj(current_params)
                    print(f"Error with params {safe_params}: Model does not support parameter setting")
                    continue
            except Exception as e:
                safe_params = _sanitize_paths_in_obj(current_params)
                print(f"Error setting params {safe_params}: {e}")
                continue
            
            try:
                # Train the model - CNN models ALWAYS need class_names (required positional argument)
                if 'CNN' in modelName:
                    model_copy.train(X_val, y_val, class_names)
                else:
                    model_copy.train(X_val, y_val)
                
                # Evaluate the model
                eval_metrics = model_copy.evaluate(X_val, y_val)
                
                # Calculate weighted score
                weighted_score = calculate_weighted_score(eval_metrics)
                
                # Store results
                result_entry = {
                    'params': current_params.copy(),
                    'metrics': eval_metrics.copy(),
                    'weighted_score': weighted_score
                }
                current_results.append(result_entry)
                
                # Update best model if this one is better
                if weighted_score > best_score:
                    best_score = weighted_score
                    best_params = current_params.copy()
                    best_model = deepcopy(model_copy)
                
                # Progress indicator
                if (i + 1) % max(1, total_combinations // 10) == 0 or i == total_combinations - 1:
                    print(f"Progress: {i+1}/{total_combinations} ({(i+1)/total_combinations*100:.1f}%) - "
                          f"Current best weighted score: {best_score:.4f}")
                
                # PERFORMANCE: Halt the process when it reaches near-perfect score
                # Avoids Overfitting!
                if best_score >= 0.93 * 0.8: # applied penalty to scores
                    print("Reached excellent score (≥0.93), stopping further evaluations.")
                    break
                    
            except Exception as e:
                # Sanitize any path-like values in current_params before printing
                safe_params = _sanitize_paths_in_obj(current_params)
                print(f"Error with params {safe_params}: {str(e)}")
                continue

            iteration_count += 1
        
        # Store results for this model-dataset combination
        tuning_results[(modelName, dataset)] = {
            'best_params': best_params,
            'best_score': best_score,
            'best_metrics': best_model.evaluate(X_val, y_val) if best_model else None,
            'all_results': current_results
        }
        
        # Store the best trained model
        model_dataset_param[(modelName, dataset)] = best_model
        
        # Print summary for this model-dataset combination
        print(f"\nBest configuration for {modelName} on {dataset}:")
        print(f"Parameters: {_sanitize_paths_in_obj(best_params)}")
        print(f"Weighted Score: {best_score:.4f}")
        if best_model:
            best_metrics = best_model.evaluate(X_val, y_val)
            for metric, value in best_metrics.items():
                print(f"  {metric}: {value:.4f}")

print(f"\n{'='*80}")
print("HYPERPARAMETER TUNING COMPLETED!")
print(f"{'='*80}")
print(f"Results summary:")
for (model_name, dataset_name), results in tuning_results.items():
    if 'error' in results:
        print(f"{model_name} on {dataset_name}: [ERROR] {results['error']}")
    else:
        safe_best_params = _sanitize_paths_in_obj(results.get('best_params'))
        print(f"{model_name} on {dataset_name}: Best weighted score = {results['best_score']:.4f}; params={safe_best_params}")


Tuning hyperparameters for Decision Tree Model on CIFAR-10
Using class names: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
Testing 1568 parameter combinations...
Reached excellent score (≥0.93), stopping further evaluations.

Best configuration for Decision Tree Model on CIFAR-10:
Parameters: {'max_depth': 18, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'gini', 'splitter': 'random'}
Weighted Score: 0.8000
  accuracy: 1.0000
  precision: 1.0000
  recall: 1.0000
  F1 (Macro): 1.0000
  F1 (Micro): 1.0000
  ROC AUC: 1.0000

Tuning hyperparameters for K-Nearest Neighbors Model on CIFAR-10
Using class names: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
Testing 336 parameter combinations...
Reached excellent score (≥0.93), stopping further evaluations.

Best configuration for K-Nearest Neighbors Model on CIFAR-10:
Parameters: {'n_neighbors': 3, 'weights': 'distance', 'algorithm': 'kd_t

Train Loss: 2.3297, Train Acc: 0.1150 (11.50%)
Val Loss: 2.3037, Val Acc: 0.1300 (13.00%)
Saved best model (val_acc=0.1300) to .cache/models/cnn_cifar.pth
Epoch [1/10], Train Loss: 2.3297, Val Loss: 2.3037

Epoch 2/10


Train Loss: 2.1525, Train Acc: 0.1975 (19.75%)
Val Loss: 2.2566, Val Acc: 0.1600 (16.00%)
Saved best model (val_acc=0.1600) to .cache/models/cnn_cifar.pth
Epoch [2/10], Train Loss: 2.1525, Val Loss: 2.2566

Epoch 3/10


Train Loss: 2.0585, Train Acc: 0.2425 (24.25%)
Val Loss: 2.1422, Val Acc: 0.2500 (25.00%)
Saved best model (val_acc=0.2500) to .cache/models/cnn_cifar.pth
Epoch [3/10], Train Loss: 2.0585, Val Loss: 2.1422

Epoch 4/10


Train Loss: 1.9620, Train Acc: 0.3300 (33.00%)
Val Loss: 2.1317, Val Acc: 0.2200 (22.00%)
Epoch [4/10], Train Loss: 1.9620, Val Loss: 2.1317

Epoch 5/10


...

Train Loss: 0.5479, Train Acc: 0.9600 (96.00%)
Val Loss: 0.7942, Val Acc: 0.8700 (87.00%)
Saved best model (val_acc=0.8700) to .cache/models/cnn_cifar.pth
Epoch [6/10], Train Loss: 0.5479, Val Loss: 0.7942

Epoch 7/10


Train Loss: 0.4438, Train Acc: 0.9750 (97.50%)
Val Loss: 0.6893, Val Acc: 0.8700 (87.00%)
Epoch [7/10], Train Loss: 0.4438, Val Loss: 0.6893

Epoch 8/10


Train Loss: 0.3275, Train Acc: 0.9850 (98.50%)
Val Loss: 0.6446, Val Acc: 0.8600 (86.00%)
Epoch [8/10], Train Loss: 0.3275, Val Loss: 0.6446

Epoch 9/10


Train Loss: 0.3036, Train Acc: 0.9950 (99.50%)
Val Loss: 0.6135, Val Acc: 0.8700 (87.00%)
Epoch [9/10], Train Loss: 0.3036, Val Loss: 0.6135

Epoch 10/10


Train Loss: 0.3274, Train Acc: 0.9925 (99.25%)
Val Loss: 0.6085, Val Acc: 0.8700 (87.00%)
Epoch [10/10], Train Loss: 0.3274, Val Loss: 0.6085

Training complete!
Best val acc: 0.8700 (87.00%)
Training history plot saved to: .cache\results\cnn_training_history_20251111_182314.png
Training history data saved to: .cache\results\cnn_training_history_20251111_182314_data.txt
Confusion matrices saved to: .cache\results\cnn_confusion_matrices_20251111_182315.png
Confusion matrix data saved to: .cache\results\cnn_confusion_matrices_20251111_182315_data.txt
Preprocessing input data for test data...
Original shape: (500, 1024)
Detected 32x32 grayscale format: reshaped to (500, 32, 32, 1)
Final tensor shape: torch.Size([500, 1, 32, 32]) (N, C, H, W)
Preprocessing input data for test data...
Original shape: (500, 1024)
Detected 32x32 grayscale format: reshaped to (500, 32, 32, 1)
Final tensor shape: torch.Size([500, 1, 32, 32]) (N, C, H, W)
Reached excellent score (≥0.93), stopping further evaluat

The tuning might be too naive since it just kept tuning till reaching the best metrics (although it already caps at 0.93). That case must be overfitting (i.e., memorizing) solely the training (seen) data. We need more sophisticated logic. But now at least we know our end-to-end flow works well during the pure tuning processes before introducing metaheuristics. 

## Train the Model

Train it with best set of parameters found.

In [ ]:
trained_best_models: dict = {}
for (modelName, datasetName), results in tuning_results.items():
    print(f"\nTraining final model for {modelName} on {datasetName} with best hyperparameters...")
    
    # Check if tuning was successful
    if 'error' in results:
        print(f"[ERROR] Skipping {modelName} on {datasetName} due to tuning error: {results['error']}")
        continue
    
    best_model = model_dataset_param.get((modelName, datasetName))
    if best_model is None:
        print(f"[ERROR] No best model found for {modelName} on {datasetName}, skipping.")
        continue
    
    # Get class names for CNN models - ALWAYS get them for CNN compatibility
    class_names = get_class_names_from_dataset(datasetName)
    
    # Retrain on full training data (train only)
    match datasetName:
        case 'CIFAR-10':
            X_train, y_train = extract_features_and_labels(CIFAR10_TRAIN, flatten_images=True)
        case 'MNIST':
            X_train, y_train = extract_features_and_labels(MNIST_TRAIN, flatten_images=True)
    if not hasattr(best_model, 'train'):
        print(f"[WARNING] Model {modelName} does not support training method. Skipping.")
        continue
    
    # Train - CNN models ALWAYS need class_names (required positional argument)
    if 'CNN' in modelName:
        best_model.train(X_train, y_train, class_names)
    else:
        best_model.train(X_train, y_train)
    print("Training completed.")
    # Save Trained Models
    trained_best_models[(modelName, datasetName)] = best_model


Training final model for Decision Tree Model on CIFAR-10 with best hyperparameters...
Training completed.

Training final model for K-Nearest Neighbors Model on CIFAR-10 with best hyperparameters...
Training completed.

Training final model for Logistic Regression Model on CIFAR-10 with best hyperparameters...


## Evaluate the Trained Best Models

In [ ]:
metrics: dict = {}
for (modelName, datasetName), best_model in trained_best_models.items():
    print(f"\nEvaluating final trained model for {modelName} on {datasetName}...")
    # Get test data
    match datasetName:
        case 'CIFAR-10':
            X_test, y_test = extract_features_and_labels(CIFAR10_TEST, flatten_images=True)
        case 'MNIST':
            X_test, y_test = extract_features_and_labels(MNIST_TEST, flatten_images=True)
    # Evaluate
    test_metrics = best_model.evaluate(X_test, y_test)
    print(f"Test Metrics for {modelName} on {datasetName}:")
    for metric, value in test_metrics.items():
        print(f"  {metric}: {value:.4f}")
        # Save metrics
    metrics[(modelName, datasetName)] = test_metrics

In [ ]:
# Find the Best Model from Weighted Sum
best_overall_score = -1
for (modelName, datasetName), test_metrics in metrics.items():
    weighted_score = calculate_weighted_score(test_metrics)
    print(f"Weighted Score for {modelName} on {datasetName}: {weighted_score:.4f}")
    if weighted_score > best_overall_score:
        best_overall_score = weighted_score
        best_model_info = (modelName, datasetName)
print(f"\nBest Overall Model: {best_model_info[0]} on {best_model_info[1]} with Weighted Score: {best_overall_score:.4f}")